Below is all the scripts I have written for lab that I found useful along with a description

In [1]:
#The 6 cells below is my creation of a datafrfame that filter for specific fasstas from a larger fasta, 
#mainly used for between species analysis of coding and noncoding regions

In [ ]:
column_name = 'af293 a1163 oerlinghausenensis fischeri lentulus fumigatiaffinis novofumigatus udagawae thermomutatus turcosus'.split()
df_final = pd.DataFrame(columns= column_name)

In [ ]:
excel_name = '/Users/alecbrown/Desktop/Rokas_Lab/Genomes_paper_1/excel_sheets/single_copy_genes.xlsx'
df = pd.read_excel(excel_name, dtype=str, na_filter=False)

In [ ]:
cds_fasta = '/Users/alecbrown/Desktop/Rokas_Lab/Genomes_paper_1/FILES_intergenic/500_bp/turcosus_intergenic_500.txt'
inFile = open(cds_fasta,'r') 

headerList = []
seqList = []
currentSeq = 'placeholder' #place holder skips the first currentseq from being ''
for line in inFile:
    if line[0] == ">":
        headerList.append(line[1:].strip())
        if currentSeq != 'placeholder':
            seqList.append(currentSeq)
   
        currentSeq = ''
        
    else:
        currentSeq += line.strip()

   
seqList.append(currentSeq)

In [ ]:
searchfor_list = []
for value in df['turcosus']: #This will be specific for df column that contains gene IDs of interest
    searchfor_list.append(value) #creates list
    
new_searchfor_list = []
for item in searchfor_list: 
    new_searchfor_list.append(item.split(','))
    
combined_list = []
for i in range(len(headerList)): 
    combined_list.append(headerList[i] +'?'+seqList[i]) #This line can be mod to have seperator between id and seq, maybe '?'
    #keep in mind, this will have > and \n for LF when written to txt file
combined_list_Series = pd.Series(combined_list)

In [ ]:
new_filter_list = []

for value in new_searchfor_list:
    working_list = []
    for items in value: 
        y = list(filter(lambda x:items in x, combined_list_Series))
        if items == '':
            working_list.append('')
        elif y: 
            working_list.append(y[0])
            
    new_filter_list.append(working_list)

In [ ]:
df_final['turcosus'] = new_filter_list

In [2]:
#The following 4 cells takes an excel sheet from above (from dataframe) and writes a fasta file for each row

In [ ]:
df = pd.read_excel('/Users/alecbrown/Desktop/Rokas_Lab/Genomes_paper_1/excel_sheets/fasta_500bp.xlsx')

In [ ]:
df = df.drop('orthogroup', axis=1)

In [ ]:
df = df.replace(r'\?','\n', regex=True)
df = df.replace(r'\[','', regex=True)
df = df.replace(r'\]','', regex=True)
df = df.replace(r"\'","", regex=True)
df = df.replace(r'\"','', regex=True)

In [ ]:
os.chdir('/Users/alecbrown/Desktop/Rokas_Lab/Genomes_paper_1/fasta/upstream_nt_500bp/')

i = 0

while i < len(df):
    #f = open('testing.txt_'.format(i), 'w')
    for row in df[i:1+i].values:
        f = open(str(i) + '_upstream.txt', 'w')
        for fasta in row:
            if fasta.rstrip() == '': #this will skip any empty values
                continue
            else:
                f.write('>' + fasta)
                f.write('\n')
    #f.close()
    i+=1

In [ ]:
#a general script of using regex to to find a pattewrn in. a larger string, and match strings to that pattern

af293_reformat = [] #doesn't have to be appended to a list

for i in af293_from_df:
    pattern = re.compile(r'E[A-Z0-9.]+')
    matches = pattern.finditer(i)
                         
    for match in matches:
        af293_reformat.append(match.group(0))

In [ ]:
#Script to calculate percent similarity given a directory of fasta files, from af293_a1163_9_9_20

import os
import pandas as pd
from Bio import pairwise2
from Bio.pairwise2 import format_alignment
import re

def percent_similarities(path):
    
    os.chdir(path)
    
    column = 'Gene_name af293:a1163'.split()
    df = pd.DataFrame(columns=column)

    for fasta in os.listdir(path):
        if 'upstream.txt' in str(fasta):
            file = str(fasta)
            #print(file)
        else:
            continue



        percentages = []
        seqs = []
        score = None
        total_len = None

        with open(file, 'r') as f:
            for line in f:
                if '>' in line and 'EDP' in line:
                    continue 
                if '>' in line and 'AFUA_' in line:
                    pattern = re.compile(r'Name=AFUA_[0-9A-Z.]+')
                    matches = pattern.finditer(line)
                    
                    for match in matches:
                        percentages.append(match.group(0)[5:])
                if '>' in line:
                    continue
                else:
                    seqs.append(line)
        if len(seqs) < 2:
            print('PROBLEMMMMMMM with this file')
            print(seqs[-1])
            continue

        for i in range(1, len(seqs)):
        #while counter < len(seqs):
            a = seqs[0]
            b = seqs[i]

            pairwise_results = pairwise2.align.globalxx(a, b)
            for item in pairwise_results:
                score = (item[2])

            #print(len(a))
            #print(len(b))
            #print(a)
            if len(a) >= len(b):
                total_len = len(a)
            else:
                total_len = len(b)


            percentages.append(score/total_len)
        df.loc[len(df.index)]=percentages

            #counter +=1
    return(df)

In [3]:
#Below gets results from a directory with AU_reesults using iqtree as a single dataframe
#from AU_test_results_9_28_20

In [ ]:
def get_au_results(path, handle):
    
    os.chdir(path)

    dct = {}
    for file in os.listdir():
        if '.iqtree' not in file:
            continue
        else:
            with open(file, 'r') as f:
                s = []
                l = []
                file_name = str(file)
                read = False 
                for line in f:
                    if '-----------------------------------------------------------------' in line:
                        read = True
                    if ':' in line:
                        read = False
                    if read == True:
                        s.append(line.strip())
                    if read == False:
                        continue 
                for i in s[:3]:
                    if '--------' in i:
                        continue
                    if i == '[]':
                        continue
                    else:
                        #print(i)
                        values = i.split()
                        for items in values:
                            if items[0] == '2':
                                dct[file_name] = f'{values[-2]}'
    #print(dct)
    column_names = 'file_name au_result'.split()
    df = pd.DataFrame(dct.items(), columns=column_names)
    return(df)

In [4]:
#Below is a function that will take a dir with codmel results from codeml and generate df with specifics
#From Curatrating codeml results files 9_10_20

In [ ]:
import os
import pandas as pd
import re

def codeml_null(path):
    '''Get information from codeml result files for further LRT test & dn/ds analysis'''
    
    gene_name_null = []
    lnl_null = []
    w_null = []
    
    for filename in os.listdir(path):
        if 'result.txt' in filename:
            #with open(filename, 'r') as f:
            with open(os.path.join(path, filename), 'r') as f:
                for line in f:
                    if '#1:' in line:
                        gene_name_null.append(line[4:-2].strip())
                    if 'lnL' in line and 'np: 20' in line:
                        #lnl_null.append(line[:-2].strip())
                        pattern = re.compile(r'-[0-9.]+')
                        matches = pattern.finditer(line)
                        
                        for match in matches:
                            lnl_null.append(match.group(0))
                    if 'omega' in line:
                        w_null.append(line[:-2].strip())
 
    
          
    column_names = 'protein_id lnl_null dn/ds_null'.split()
    
    df = pd.DataFrame(columns=column_names)
          
    df['protein_id'] = gene_name_null
    df['lnl_null'] = lnl_null
    df['dn/ds_null'] = w_null
    
    return(df)

In [ ]:
def codeml_alt(path):
    '''Get information from codeml result files for further LRT test & dn/ds analysis'''
    
    gene_name_alt = []
    lnl_alt = []
    w_alt = []
    
    for filename in os.listdir(path):
        if 'result.txt' in filename:
            #with open(filename, 'r') as f:
            with open(os.path.join(path, filename), 'r') as f:
                for line in f:
                    if '#1:' in line:
                        gene_name_alt.append(line[4:-2].strip())
                    if 'lnL' in line and 'np: 21':
                        pattern = re.compile(r'-[0-9.]+')
                        matches = pattern.finditer(line)
                        
                        for match in matches:
                            lnl_alt.append(match.group(0)[:-2].strip())
                    if '(dN/dS) for branches:' in line:
                        w_alt.append(line[:-2].strip())
 
    
          
    column_names = 'protein_id lnl_alt dn/ds_alt'.split()
    
    df = pd.DataFrame(columns=column_names)
          
    df['protein_id'] = gene_name_alt
    df['lnl_alt'] = lnl_alt
    df['dn/ds_alt'] = w_alt
    
    return(df)

In [5]:
#The next 3 cells were used to generate dot plots for some RNA seq data from excel data, from figures RNA seq 9_13_20

In [ ]:
import pandas as pd
import os
import seaborn as sns
from matplotlib import pyplot as plt
import numpy as np

plt.style.use('seaborn-colorblind')

plt.figure(figsize=(16,9), dpi=100)

x = df['noncoding']
y = df['coding']

plt.scatter(x,y, s=50, c='skyblue', edgecolors='black', alpha=0.20)
plt.xlabel ('Non-coding region similarity (up to 500bp upstream transcription start site)')
plt.ylabel('Coding region similarity (protein sequence)')
plt.title('Seqeunce similarity of coding and noncoding regions for single copy orthologous genes. Af293:A1163')

plt.yticks(np.arange(0, 1.1, 0.1))
plt.xticks(np.arange(0, 1.1, 0.1))
plt.grid(True)



#fig1 = plt.gcf()

#fig1.savefig('test.png', dpi=100)
plt.tight_layout

#plt.savefig('af293_a1163_pairwise.png')
plt.show()
plt.close()

In [ ]:
def make_graphs(path, t, d): #path=path to RNA seq data, t = title from excel sheet of path, d = original percentages excel sheet
    '''Take a previously made scatterplot of id percentage and add a list of data to it, in a new color'''
    
    df_rnaseq = pd.read_excel(path)
    
    df_lst = list(df_rnaseq[t].dropna())
    
    #return(df_lst)
    
    gene_names = []
    for i in df_lst:
        gene_names.append(f'AFUA_{i[3:].upper()}')
        
    #return(gene_names)
    
    df = pd.read_excel(d)
    
    dct_coding = dict(zip(df['gene_name'], df['coding']))
    dct_noncoding = dict(zip(df['gene_name'], df['noncoding']))
    
    plt.style.use('seaborn-colorblind')

    plt.figure(figsize=(16,9), dpi=100)

    x = df['noncoding']
    y = df['coding']
    
    a = []
    b = []
    
    for i in gene_names:
        if i in dct_coding.keys():
            a.append(dct_coding.get(i))
            
    for i in gene_names:
        if i in dct_noncoding.keys():
            b.append(dct_noncoding.get(i))

    plt.scatter(x,y, s=50, c='skyblue', edgecolors='black', alpha=0.75, label='Genomic background')
    plt.scatter(b,a, s=50, c='orange', edgecolors='black', alpha=1.0, label = 'Differently expressed gene')
    plt.xlabel ('Non-coding region similarity')
    plt.ylabel('Coding region similarity')
    plt.title(f'Seqeunce similarity of coding and noncoding regions for single copy orthologous genes: delta_crza_{t}')

    plt.yticks(np.arange(0, 1.1, 0.1))
    plt.xticks(np.arange(0, 1.1, 0.1))
    plt.grid(True)
    
    plt.legend(loc = 'lower right')

    plt.tight_layout

    plt.savefig(f'RNA_Seq_delta_crza_{t}.png')
    return(plt.show())
    #plt.close()

In [ ]:
#Here, let's compare between wildtype and delta crza based on type
#Let's add a legend here as well

def compare_graphs(path_wt, path_crza, t, d): #path=path to RNA seq data, t = title from excel sheet of path, d = original percentages excel sheet
    '''Take a previously made scatterplot of id percentage and add a list of data to it, in a new color'''
    
    df_rnaseq = pd.read_excel(path_wt)
    df_deltacrza = pd.read_excel(path_crza)
    
    df_lst = list(df_rnaseq[t].dropna())
    df_lst_2 = list(df_deltacrza[t].dropna())
    
    #return(df_lst)
    
    gene_names = []
    for i in df_lst:
        gene_names.append(f'AFUA_{i[3:].upper()}')
        
    gene_names_delta = []
    for i in df_lst_2:
        gene_names_delta.append(f'AFUA_{i[3:].upper()}')
        
    #return(gene_names)
    
    df = pd.read_excel(d)
    
    dct_coding = dict(zip(df['gene_name'], df['coding']))
    dct_noncoding = dict(zip(df['gene_name'], df['noncoding']))
    
    plt.style.use('seaborn-colorblind')

    plt.figure(figsize=(16,9), dpi=100)

    x = df['noncoding']
    y = df['coding']
    
    a = []
    b = []
    c = []
    d = []
    
    for i in gene_names:
        if i in dct_coding.keys():
            a.append(dct_coding.get(i))
            
    for i in gene_names:
        if i in dct_noncoding.keys():
            b.append(dct_noncoding.get(i))
            
    for i in gene_names_delta:
        if i in dct_noncoding.keys():
            c.append(dct_noncoding.get(i))

    for i in gene_names_delta:
        if i in dct_coding.keys():
            d.append(dct_coding.get(i))
            
    #plt.scatter(x,y, s=50, c='skyblue', edgecolors='black', alpha=0.5)
    plt.scatter(b,a, s=50, c='red', edgecolors='black', alpha=0.5, label = 'Wildtype')
    plt.scatter(c,d, s=50, c='blue', edgecolors='black', alpha=0.5, label = 'Delta crza')
    plt.xlabel ('Non-coding region similarity')
    plt.ylabel('Coding region similarity')
    plt.title(f'Seqeunce similarity comparison of coding and noncoding regions between wildtype and delta crza: {t}')

    plt.yticks(np.arange(0, 1.1, 0.1))
    plt.xticks(np.arange(0, 1.1, 0.1))
    plt.grid(True)

    plt.scatter([],[], s=50, c = 'purple', edgecolors='black', label = 'Common gene')
    plt.legend(loc = 'lower right')

    #fig1 = plt.gcf()

    #fig1.savefig('test.png', dpi=100)
    plt.tight_layout

    plt.savefig(f'RNA_Seq_{t}_comparison.png')
    return(plt.show())
    plt.close()

In [6]:
#Below takes FIMO results .tsv files and generates df comparing crzA location and difs between af293 & a1163
#from fimo_data 11-21-20

In [ ]:
def fimo_data(path):
    
    af293_names = []
    a1163_names = []


    number_dif = []
    location_dif = []
    
    os.chdir(path)
    for tsv_file in os.listdir():
        if '.tsv' in tsv_file:
        
            df_tsv = pd.read_csv(tsv_file, sep='\t').dropna()
            
            af293_lst = []
            a1163_lst = []
            
            if 'sequence_name' not in df_tsv.columns:
                continue
            else:
                
                seq_names = list(df_tsv['sequence_name'].unique())
                
                if seq_names == []:
                    continue 

                df_test = pd.read_csv(tsv_file, sep='\t').dropna()

                for i in seq_names:
                    if 'AFUA' in i:
                        #af293_names.append(i)
                        af293_names.append(i)

                        af293_df = df_test[df_test['sequence_name'].isin(af293_names)]
                        af293_start = list(af293_df['start'])
                        af293_stop = list(af293_df['stop'])

                        for i in af293_start:
                            af293_lst.append(i)
                        for i in af293_stop:
                            af293_lst.append(i)

                    else:
                        #a1163_names.append(i)
                        a1163_names.append(i)

                        a1163_df = df_test[df_test['sequence_name'].isin(a1163_names)]
                        a1163_start = list(a1163_df['start'])
                        a1163_stop = list(a1163_df['stop'])  

                        for i in a1163_start:
                            a1163_lst.append(i)
                        for i in a1163_stop:
                            a1163_lst.append(i)
                            
                    if len(seq_names) != 2:
                        for i in seq_names:
                            if 'AFUA' in i:
                                a1163_names.append('None')
                            if 'AFUB' in i:
                                af293_names.append('None')

                af293_lst.sort()
                a1163_lst.sort()

                if len(af293_lst) == len(a1163_lst):
                    number_dif.append(False)
                else:
                    number_dif.append(True)


                if af293_lst == a1163_lst:
                    location_dif.append(False)
                else:
                    location_dif.append(True)
    
    print(f'len of af293_names: {len(af293_names)}')
    print(f'len of a1163_names: {len(a1163_names)}')
    print(f'len of nums dif: {len(number_dif)}')
    print(f'len of location dif: {len(location_dif)}')

    cols = 'af293_name a1163_name Differences_in_number_of_crza_sites Differencces_in_crza_site_location'.split()
    df_final = pd.DataFrame(columns=cols)

    df_final['af293_name'] = af293_names
    df_final['a1163_name'] = a1163_names
    df_final['Differences_in_number_of_crza_sites'] = number_dif
    df_final['Differencces_in_crza_site_location'] = location_dif
    

    return(df_final)

In [7]:
#The following generate a violin. plot for dn/ds analysis from results excel sheet, I give an example dir as well
#From LTR_for_codeml_and_graphs

In [ ]:
os.chdir('/Users/alecbrown/Desktop/Rokas_Lab/Genomes_paper_1/dnds_results/')

In [ ]:
from matplotlib import pyplot as plt

sns.set(style="whitegrid")

plt.figure(figsize=(16,9), dpi=120)

ax = sns.violinplot(data = [data_no_outlier, gdov_data], palette = ['cyan', 'coral'])
ax.set_ylabel('dN/dS')
#ax.set_xlabel('Group')
ax.set_xticklabels(['Across Phylogeny','Genetic Determinants of Virulence'])

plt.legend(title='Average dN/dS', loc='upper right', labels=['A.P dN/dS average is: 0.189', 'G.D.O.V dN/dS average is: 0.147'])

plt.tight_layout

plt.savefig('dn_ds_coding_average.png')          

#plt.savefig(ax)
plt.show(ax)

In [8]:
#Below was the start of reformating codeml ctl files from a sample file
#This and many other reformatting projects can be found orthofinder_make_fasta_10_29_20

In [ ]:
os.chdr('/Users/alecbrown/Desktop/Rokas_Lab/Genomes_paper_1/fasta/cds/cds_nt_10_29_20_reformat/')
for files in os.listdir():
    
    for i in file_head:
        if f'{str(i)}_fasta_cds.txt.pml' in files:

            with open('ctl_template.txt', 'r') as f, open(f'{i}.ctl', 'w') as nf:
                for line in f:
                    if 'seqfile' in line:
                        new_line1 = line.replace('replace_me', f'{i}.pml')
                        nf.write(new_line1)
                    if 'outfile =' in line:
                        new_line = line.replace('replace_me', f'{i}_result.txt')
                        nf.write(new_line)
                    if 'replace_me' in line:
                        continue
                    else:
                        nf.write(line)

In [ ]:
#make ctl files for au_rejected 
os.chdir('/Users/alecbrown/Desktop/Rokas_Lab/Genomes_paper_1/fasta/cds/codeml_branch_au_rejected_1_15_20/')
for i in os.listdir():
    
    if '.pml' in i:
        a,b = i.split('.')
        #if f'{str(i)}.pml' in files:

        with open('ctl_template.txt', 'r') as f, open(f'{a}.ctl', 'w') as nf:
            for line in f:
                if 'seqfile' in line:
                    new_line1 = line.replace('replace_me', f'{a}.pml')
                    nf.write(new_line1)
                if 'treefile =' in line:
                    new_line = line.replace('replace_me', f'{a}_tree.txt')
                    nf.write(new_line)
                if 'outfile =' in line:
                    new_line = line.replace('replace_me', f'{a}_result.txt')
                    nf.write(new_line)
                if 'replace_me' in line:
                    continue
                else:
                    nf.write(line)

In [9]:
#from percent_sim_check_11_5_20:
#***use the pandas merge to merge two dataframes***

In [ ]:
df_new = pd.merge(df_ana, df_fimo, how='left', left_on=['af293_gene', 'a1163_gene'], right_on = ['af293_name', 'a1163_name'])

In [10]:
#Reformat fasta files and write new files from reformat_fasta_9_13_20

In [ ]:
num = 5215
path = '/Users/alecbrown/Desktop/Rokas_Lab/Genomes_paper_1/fasta/upstream_nt_1.5kb/'
os.chdir(path)

for i in range(0,num):
    with open(f'{i}_fasta.txt', 'r') as f, open(f'{i}_fasta_reformat.txt', 'w') as nf:
        fasta_id = []
        seqs = []
        for line in f:
            if '>' in line:
                if 'AFUA_' in line:
                    pattern = re.compile(r'Name=AFUA_[0-9A-Z]+')
                    matches = pattern.finditer(line)
                                         
                    for m in matches:
                        fasta_id.append(m.group(0)[5:])
                if 'AFUB_' in line:
                    pattern = re.compile(r'Name=AFUB_[0-9]+')
                    matches = pattern.finditer(line)
                                         
                    for m in matches:
                        fasta_id.append(m.group(0)[5:])
                                         
                if 'ID=A_oerling' in line:
                    pattern = re.compile(r'ID=A_oerling_[A-Z0-9_]+')
                    matches = pattern.finditer(line)
                                         
                    for m in matches:
                        fasta_id.append(m.group(0)[3:])
                                         
                if 'NFIA_' in line:
                    pattern = re.compile(r'Name=NFIA_[0-9]+')
                    matches = pattern.finditer(line)
                                         
                    for m in matches:
                        fasta_id.append(m.group(0)[5:])
                                         
                if 'ALT_' in line:
                    pattern = re.compile(r'Name=ALT_[0-9]+')
                    matches = pattern.finditer(line)
                                         
                    for m in matches:
                        fasta_id.append(m.group(0)[5:])
                            
                if '.g' in line:
                    pattern = re.compile(r'.g[0-9]+')
                    matches = pattern.finditer(line)
                                         
                    for m in matches:
                        fasta_id.append(m.group(0)[1:])
                                         
                if 'P174DRAFT' in line:
                    pattern = re.compile(r'Name=P174DRAFT_[0-9]+')
                    matches = pattern.finditer(line)
                                         
                    for m in matches:
                        fasta_id.append(m.group(0)[5:])
                                         
                if 'AUD_' in line:
                    pattern = re.compile(r'Name=AUD_[0-9]+')
                    matches = pattern.finditer(line)
                                         
                    for m in matches:
                        fasta_id.append(m.group(0)[5:])
                                         
                if 'CDV56_' in line:
                    pattern = re.compile(r'locus_tag=CDV56_[0-9]+')
                    matches = pattern.finditer(line)
                                         
                    for m in matches:
                        fasta_id.append(m.group(0)[10:])
                                         
                if 'CFD26_' in line:
                    pattern = re.compile(r'locus_tag=CFD26_[0-9]+')
                    matches = pattern.finditer(line)
                                         
                    for m in matches:
                        fasta_id.append(m.group(0)[10:])
                        
            else:
                seqs.append(line[:-2])
                                         
            #print(seqs)
                #nf.write(line)
        #print(len(seqs))
        #print('--------BREAK-------')
        #print(seqs)
        if len(seqs) != len(fasta_id):
            print(len(seqs))
            print(len(fasta_id))
            continue
            
        else:
            #print(len(seqs))
            dct = dict(zip(fasta_id, seqs))
            
            for k,v in dct.items():
                nf.write('>' + k + '\n')
                nf.write(v + '\n')

In [ ]:
#Following 3 scripts for reformating found in reformat_for_AU_test_9_25_20

In [ ]:
def reformat_trees(path, old_tf, new_tf, num):
    
    os.chdir(path)
    
    for i in range(0,num):
        if f'{str(i)}{old_tf}' not in os.listdir():
            continue
        with open(f'{str(i)}{old_tf}', 'r') as oldfile, open(f'{str(i)}{new_tf}', 'w') as newfile:
            dct = {'Aspergillus_fumigatus_af293':None, 'Aspergillus_fumigatus_a1163':None, 
                   'Aspergillus_oerlinghausenensis':None, 
                  'Aspergillus_fischeri':None, 'Aspergillus_lentulus':None, 'Aspergillus_novofumigatus':None, 
                   'Aspergillus_fumigatiaffinis':None, 'Aspergillus_udagawae':None, 'Aspergillus_turcosus':None, 
                   'Aspergillus_thermomutatus':None }

            for line in oldfile:
                if 'AFUA_' in line:
                    pattern = re.compile(r'AFUA_[0-9A-Z]+')
                    matches = pattern.finditer(line)

                    for m in matches:
                        dct['Aspergillus_fumigatus_af293'] = m.group(0)

                    pattern_2 = re.compile(r'AFUB_[0-9]+')
                    matches_2 = pattern_2.finditer(line)

                    for m in matches_2:
                        dct['Aspergillus_fumigatus_a1163'] = m.group(0)

                    pattern_3 = re.compile(r'A_oerling_[0-9A-Z_]+')
                    matches_3 = pattern_3.finditer(line)

                    for m in matches_3:
                        dct['Aspergillus_oerlinghausenensis'] = m.group(0)

                    pattern_4 = re.compile(r'NFIA_[0-9]+')
                    matches_4 = pattern_4.finditer(line)

                    for m in matches_4:
                        dct['Aspergillus_fischeri'] = m.group(0)

                    pattern_5 = re.compile(r'ALT_[0-9]+')
                    matches_5 = pattern_5.finditer(line)

                    for m in matches_5:
                        dct['Aspergillus_lentulus'] = m.group(0)


                    pattern_7 = re.compile(r'P174DRAFT_[0-9]+')
                    matches_7 = pattern_7.finditer(line)

                    for m in matches_7:
                        dct['Aspergillus_novofumigatus'] = m.group(0)

                    pattern_8 = re.compile(r'AUD_[0-9]+')
                    matches_8 = pattern_8.finditer(line)

                    for m in matches_8:
                        dct['Aspergillus_udagawae'] = m.group(0)

                    pattern_9 = re.compile(r'CDV56_[0-9]+')
                    matches_9 = pattern_9.finditer(line)

                    for m in matches_9:
                        dct['Aspergillus_thermomutatus'] = m.group(0)

                    pattern_10 = re.compile(r'CFD26_[0-9]+')
                    matches_10 = pattern_10.finditer(line)

                    for m in matches_10:
                        dct['Aspergillus_turcosus'] = m.group(0)
                        
                    pattern_11 = re.compile(r'\([g0-9]+')
                    matches_11 = pattern_11.finditer(line)

                    for m in matches_11:
                        dct['Aspergillus_fumigatiaffinis'] = m.group(0)[1:]
                        
                    pattern_12 = re.compile(r',g[0-9]+')
                    matches_12 = pattern_12.finditer(line)

                    for m in matches_12:
                        dct['Aspergillus_fumigatiaffinis'] = m.group(0)[1:]

                    new_line = line
                    for k,v in dct.items():
                        if v == None:
                            continue
                        if v in line:
                            new_line = new_line.replace(v,k) #need a way to save the line each time we do this
                    newfile.write(new_line)


In [ ]:
def reformat_mafft(path, old_tf, new_tf, num):
    
    os.chdir(path)
    
    for i in range(0,num):
        if f'{str(i)}{old_tf}' not in os.listdir():
            continue
        with open(f'{str(i)}{old_tf}', 'r') as oldfile, open(f'{str(i)}{new_tf}', 'w') as newfile:
            fasta_id = []
            seqs = []
            for line in oldfile:
                if '>' in line:
                    if 'AFUA_' in line:
                        new_line = line.replace(line, 'Aspergillus_fumigatus_af293')
                        newfile.write('>'+ new_line + '\n')
                    if 'AFUB_' in line:
                        new_line = line.replace(line, 'Aspergillus_fumigatus_a1163')
                        newfile.write('>'+ new_line + '\n')
                    if 'A_oerling' in line:
                        new_line = line.replace(line, 'Aspergillus_oerlinghausenensis')
                        newfile.write('>'+ new_line + '\n')   
                    if 'NFIA_' in line:
                        new_line = line.replace(line, 'Aspergillus_fischeri')
                        newfile.write('>'+ new_line + '\n')
                    if 'ALT_' in line:
                        new_line = line.replace(line, 'Aspergillus_lentulus')
                        newfile.write('>'+ new_line + '\n') 
                    if '>g' in line:
                        new_line = line.replace(line, 'Aspergillus_fumigatiaffinis')
                        newfile.write('>'+ new_line + '\n') 
                    if 'P174DRAFT' in line:
                        new_line = line.replace(line, 'Aspergillus_novofumigatus')
                        newfile.write('>'+ new_line + '\n') 
                    if 'AUD_' in line:
                        new_line = line.replace(line, 'Aspergillus_udagawae')
                        newfile.write('>'+ new_line + '\n') 
                    if 'CDV56_' in line:
                        new_line = line.replace(line, 'Aspergillus_thermomutatus')
                        newfile.write('>'+ new_line + '\n') 
                    if 'CFD26_' in line:
                        new_line = line.replace(line, 'Aspergillus_turcosus')
                        newfile.write('>'+ new_line + '\n') 

                else:
                    newfile.write(line[:-2]+'\n')

In [ ]:
def new_files_for_au(path, handle, newfile, num):
    
    os.chdir(path)
    
    for i in range(0,num):
        if f'{str(i)}{handle}' not in os.listdir():
            continue
        else:
            with open(f'{str(i)}{handle}', 'r') as oldfile, open(f'{str(i)}{newfile}', 'w') as nf:
                for line in oldfile:
                    nf.write(line)

In [ ]:
#Making some fasta files for af293 and a1163

def make_some_fastas(path, num):

    #num = 5215
    #path = '/Users/alecbrown/Desktop/Rokas_Lab/Genomes_paper_1/fasta/upstream_nt_1.5kb/'
    os.chdir(path)

    for i in range(0,num):
        with open(f'{i}_upstream.txt', 'r') as f, open(f'{i}_fasta.txt', 'w') as nf:
            fasta_id = []
            seqs = []
            for line in f:
                if '>' in line:
                    if 'AFUA_' in line:
                        pattern = re.compile(r'Name=AFUA_[0-9A-Z]+')
                        matches = pattern.finditer(line)

                        for m in matches:
                            fasta_id.append(m.group(0)[5:])
                    if 'AFUB_' in line:
                        pattern = re.compile(r'Name=AFUB_[0-9]+')
                        matches = pattern.finditer(line)

                        for m in matches:
                            fasta_id.append(m.group(0)[5:])

                else:
                    seqs.append(line[:-2])

            if len(seqs) != len(fasta_id):
                print(len(seqs))
                print(len(fasta_id))
                continue

            else:
                #print(len(seqs))
                dct = dict(zip(fasta_id, seqs))

                for k,v in dct.items():
                    nf.write('>' + k + '\n')
                    nf.write(v + '\n')

In [11]:
#Make sequence logos - found in make_logos

In [ ]:
#More reformatting protein ids and gene ids in orthofinder_make_fasta_10_29_20

In [12]:
#the following gives a column by column snps between sequences, MUCH MORE in snps_columns notebook 

In [ ]:
nums = []
snps = []
remove_commas = []

for i in range(len(fumigatus_seq)):
    fum_cols = ','.join(fumigatus_seq[i])
    fis_cols = ','.join(fischeri_seq[i])
    
    if len(fumigatus_seq[i]) <= len(fischeri_seq[i]):
        index = len(','.join(fumigatus_seq[i]))
    else:
        index = len(','.join(fischeri_seq[i]))
        
    
    for j in range(index):
        if (fum_cols[j] and fis_cols[j]) == ',':
            remove_commas.append('')
        
        else:
            if (fum_cols[j] == fis_cols[j]) and fum_cols[j] != ',':
                nums.append(1) 
            else:
                nums.append(0)
      
        
    snps.append(nums)
    nums = []

In [ ]:
#Example of how to copy files from one dir to another 
import shutil
for i in os.listdir():
    if i in handle:
        shutil.copy(i, '/Users/alecbrown/Desktop/Rokas_Lab/Genomes_paper_1/Trees/Noncoding_500bp/RF_AU_rejected/')

In [1]:
#Get robinsons foulds results from phykikt into a df

In [ ]:
def get_rf_results(path):
    name, rf, rf_n = [], [], []
    os.chdir(path)
    for i in os.listdir():
        if "result.txt" in i:
            name.append(i)
            with open(i, 'r') as f:
                for line in f:
                    nl = line.strip()
                    nl = nl.replace('\t', '_')
                    a,b = nl.split('_')
                    rf.append(int(a))
                    rf_n.append(float(b))
                    
    heads = 'file_name rf rf_normalized'.split()
    df = pd.DataFrame(columns=heads)
    df['file_name'], df['rf'], df['rf_normalized'] = name, rf, rf_n
    return df

In [2]:
#1-11-20: The following makes a dct for mafft file with file handle as key and af293 gene name as value

In [ ]:
def make_dct(path):
    
    os.chdir(path)
    dct = {}
    
    for i in os.listdir():
        if '.txt' in i:
            a,b,c = i.split('_')
            
            with open(i, 'r') as f:
                for line in f:
                    if '>' in line and 'AFUA' in line:
                        z = line[1:-1]
                        
        dct[a] = z
        
    return dct

In [ ]:
#Script to seperate txt file by line, by word, by character

import os

def wc(file_path):
    """Takes an absolute file path/name, calculates the number of
       lines/words/chars, and returns a string of these numbers + file, e.g.:
       3 12 60 /tmp/somefile
       (both tabs and spaces are allowed as separator)"""

        with open(file_path, 'r') as f:
            content = f.read() #takes files and places into txt in python 
            num_lines = len(content.splitlines()) # split text by newline
            num_words = len(content.split()) #split text by empty space to get words
            num_chars = len(context)
            
            
    return (f'{num_lines}\t{num_words}\t{num_chars}')

In [ ]:
#reformat pml files so that they are numbered instead

def reformat_pml(path, old_tf, new_tf, num):
    
    os.chdir(path)
    
    for i in range(0,num):
        if f'{str(i)}{old_tf}' not in os.listdir():
            continue
        with open(f'{str(i)}{old_tf}', 'r') as oldfile, open(f'{str(i)}{new_tf}', 'w') as newfile:
            fasta_id = []
            seqs = []
            for line in oldfile:
                if 'AFUA_' in line:
                    new_line = line.replace(line, '1')
                    newfile.write(new_line + '\n')
                    continue
                if 'AFUB_' in line:
                    new_line = line.replace(line, '2')
                    newfile.write(new_line + '\n')
                    continue
                if 'CBS' in line:
                    new_line = line.replace(line, '3')
                    newfile.write(new_line + '\n')
                    continue
                if 'NFIA_' in line:
                    new_line = line.replace(line, '4')
                    newfile.write(new_line + '\n')
                    continue
                if 'ALT_' in line:
                    new_line = line.replace(line, '5')
                    newfile.write(new_line + '\n') 
                    continue
                if 'CNMC' in line:
                    new_line = line.replace(line, '6')
                    newfile.write(new_line + '\n') 
                    continue
                if 'P174DRAFT' in line:
                    new_line = line.replace(line, '7')
                    newfile.write(new_line + '\n') 
                    continue
                if 'AUD_' in line:
                    new_line = line.replace(line, '8')
                    newfile.write(new_line + '\n') 
                    continue
                if 'CDV56_' in line:
                    new_line = line.replace(line, '9')
                    newfile.write(new_line + '\n') 
                    continue
                if 'CFD26_' in line:
                    new_line = line.replace(line, '10')
                    newfile.write(new_line + '\n') 
                    continue

                else:
                    newfile.write(line)

In [ ]:
#script to get codmel result into a df from branch test
def get_codeml_results(path):
    
    af293_name = []
    a1163_name = []
    lnl = []
    af293_omega = []
    a1163_omega = []
    fumigatus_omega = []
    
    os.chdir(path)
    
    for file in os.listdir():
        if '.txt' in file:
            with open(file, 'r') as f:
                for line in f:
                    if '#1:' in line and 'AFUA':
                        af293_name.append(line[3:].strip()) 
                
                    if '#2:' in line and 'AFUB':
                        a1163_name.append(line[3:].strip()) 
                        
                    if 'lnL' in line:
                        total = line.split()
                        lnl.append(float(total[4]))
                        
                    if '17..1' in line and '0.' in line:
                        total = line.split()
                        af293_omega.append(float(total[4]))
    
                    if '17..2' in line and '0.' in line:
                        total = line.split()
                        a1163_omega.append(float(total[4]))
                    
                    if '16..17' in line and '0.' in line:
                        total = line.split()
                        fumigatus_omega.append(float(total[4]))
                        
        if len(af293_name) != len(af293_omega):
            af293_omega.append()
            break
    
    
    column_name = 'af293_name a1163_name af293_omega a1163_omega fumigatus_omega lnl_value'.split()

    df = pd.DataFrame(columns=column_name)
    
    df['af293_name'] = af293_name
    df['a1163_name'] = a1163_name
    df['af293_omega'] = af293_omega
    df['a1163_omega'] = a1163_omega
    df['fumigatus_omega'] = fumigatus_omega
    df['lnl_value'] = lnl
    
    return(df)

In [ ]:
def make_hyphy_batch(path):
    
    os.chdir(path)
    for i in os.listdir():

        if '_pml.txt' in i:
            a,b = i.split('_')
            #if f'{str(i)}.pml' in files:

            with open('template_hyphy.bf', 'r') as f, open(f'{a}_hyphy.bf', 'w') as nf:
                for line in f:
                    if 'replace_me_pml' in line:
                        new_line1 = line.replace('replace_me_pml', f'{path}{a}_pml.txt')
                        nf.write(new_line1)
                    if 'replace_me_fasta' in line:
                        new_line = line.replace('replace_me_fasta', f'{path}{a}_fasta.txt')
                        nf.write(new_line)
                    if 'replace_me' in line:
                        continue
                    else:
                        nf.write(line)

In [ ]:
#required externally made dct, from hyphy noncoding au rejected 
def au_rejected_make_hyphy_batch(path):
    
    os.chdir(path)
    for i in os.listdir():

        if '_pml.txt' in i:
            a,b = i.split('_')
            #if f'{str(i)}.pml' in files:

            with open('template_hyphy.bf', 'r') as f, open(f'{a}_hyphy.bf', 'w') as nf:
                for line in f:
                    if 'replace_me_pml' in line:
                        new_line1 = line.replace('replace_me_pml', f'{path}{a}_pml.txt')
                        nf.write(new_line1)
                    if 'replace_me_tree' in line:
                        new_line1 = line.replace('replace_me_tree', f'{path}{a}_tree.txt')
                        nf.write(new_line1)
                    if 'replace_me_node' in line:
                        if i in nodes_dct.keys():
                            
                            new_line1 = line.replace('replace_me_node', nodes_dct[i])
                            nf.write(new_line1)
                    if 'replace_me_fasta' in line:
                        new_line = line.replace('replace_me_fasta', f'{path}{a}_fasta.txt')
                        nf.write(new_line)
                    if 'replace_me' in line:
                        continue
                    else:
                        nf.write(line)

In [ ]:
def get_hyphy_results(path):
    
    file_name = []
    lnl_null = []
    lnl_alt = []
    lrt = []
    p_value = []
    zeta0_null = []
    zeta1_null = []
    zeta0_alt = []
    zeta1_alt = []
    zeta2_alt = []
    
    os.chdir(path)
    
    for file in os.listdir():
        if '.result' in file:
            l = []
            s = []
            with open(file, 'r') as f:
                content = f.read()
                if 'zeta0' not in content:
                    continue
                else:
                    file_name.append(file)
                    
                    for line in content.splitlines():
                        if 'Lk' in line:
                            total = line.split()
                            lnl_null.append(float(total[3]))
                            lnl_alt.append(float(total[7]))
                        if 'LRT' in line and 'p' not in line:
                            total = line.split()
                            lrt.append(float(total[2]))
                        if 'LRT' in line and 'p' in line:
                            t = line.split()
                            p_value.append(float(t[3]))
                        if 'zeta0' in line:
                            z0 = (line.strip().split())
                            l.append(float(z0[-1]))
                        if 'zeta1' in line:
                            z1 = (line.strip().split())
                            s.append(float(z1[-1]))
                        if 'zeta2' in line:
                            total = line.split()
                            zeta2_alt.append(float(total[-1]))
                
                zeta0_null.append(l[0])
                zeta0_alt.append(l[-1])
                zeta1_null.append(s[0])
                zeta1_alt.append(s[-1])
        
    print(len(file_name))
    print(len(lnl_alt))
    column_name = 'file_name lnl_null lnl_alt lrt p_value zeta0_null zeta1_null zeta0_alt zeta1_alt zeta2_alt'.split()

    df = pd.DataFrame(columns=column_name)
    
    df['file_name'] = file_name
    df['lnl_null'] = lnl_null
    df['lnl_alt'] = lnl_alt
    df['lrt'] = lrt
    df['p_value'] = p_value
    df['zeta0_null'] = zeta0_null
    df['zeta1_null'] = zeta1_null
    df['zeta0_alt'] = zeta0_alt
    df['zeta1_alt'] = zeta1_alt
    df['zeta2_alt'] = zeta2_alt
    
    return(df)

In [ ]:
#The following was used to get synteny upstream genes from SCO gene list

In [ ]:
#for oerl: 

dct_num_key= {}
dct_num_values = {}
with open('A_oerlinghausenensis_CBS139183_genomic.gff', 'r') as f:
    c = 0
    for line in f:
        if 'gene' in line:
            x = line.split(';')
            #print(x)
            for data in x:
                if 'ID=' in data and 'mRNA' not in data:
                    z = data.split('\t')
                    if 'ID=NODE' in z[8]:
                        continue
                    else:
                        dct_num_values[z[8]] = c
                        dct_num_key[c] = z[8]
                        c +=1
                        
#for fischeri
dct_num_key= {}
dct_num_values = {}
with open('A_fischeri_NRRL181_genomic.gff', 'r') as f:
    c = 0
    for line in f:
        if 'gene' in line:
            sections = line.split('\t')
            if 'gene' in sections[2] and 'NFIA_' in sections[8]:
                sections_2 = sections[8].split(';')
                dct_num_key[c] = sections_2[2][5:]
                dct_num_values[sections_2[2][5:]] = c
                c +=1
                
#for lentulus
dct_num_key= {}
dct_num_values = {}
with open('A_lentulus_IFM54703_genomic.gff', 'r') as f:
    c = 0
    for line in f:
        if 'gene' in line:
            sections = line.split('\t')
            if 'gene' in sections[2] and 'ALT_' in sections[8]:
                sections_2 = sections[8].split(';')
                dct_num_key[c] = sections_2[1][5:]
                dct_num_values[sections_2[1][5:]] = c
                c +=1
                
#novo
dct_num_key= {}
dct_num_values = {}
with open('A_novofumigatus_IBT16806_genomic.gff', 'r') as f:
    c = 0
    for line in f:
        if 'gene' in line:
            sections = line.split('\t')
            if 'gene' in sections[2] and 'P174DRAFT_' in sections[8]:
                sections_2 = sections[8].split(';')
                dct_num_key[c] = sections_2[2][5:]
                dct_num_values[sections_2[2][5:]] = c
                c +=1
                
#udagawae
dct_num_key= {}
dct_num_values = {}
with open('A_udagawae_IFM46973_genomic.gff', 'r') as f:
    c = 0
    for line in f:
        if 'gene' in line:
            sections = line.split('\t')
            if 'gene' in sections[2] and 'AUD_' in sections[8]:
                sections_2 = sections[8].split(';')
                dct_num_key[c] = sections_2[1][5:]
                dct_num_values[sections_2[1][5:]] = c
                c +=1
                
#Let's try again for thermomutatus
dct_num_key= {}
dct_num_values = {}
with open('A_thermomutatus_HMRAF39_genomic.gff', 'r') as f:
    c = 0
    for line in f:
        if 'gene' in line:
            sections = line.split('\t')
            if 'gene' in sections[2]:
                sections_2 = sections[8].split(';')
                for info in sections_2:
                    if 'CDV56' in sections_2[2]:
                        if sections_2[2][5:] in dct_num_key.values():
                            continue
                        else:
                            dct_num_key[c] = sections_2[2][5:]
                            dct_num_values[sections_2[2][5:]] = c
                            c +=1
                    else:
                        if 'CDV56' in sections_2[7]:
                            if sections_2[7][10:] in dct_num_key.values():
                                continue
                            else:
                                dct_num_key[c] = sections_2[7][10:]
                                dct_num_values[sections_2[7][10:]] = c
                                c +=1
                                
#turcosus
dct_num_key= {}
dct_num_values = {}
with open('A_turcosus_HMRAF1038_genomic.gff', 'r') as f:
    c = 0
    for line in f:
        if 'gene' in line:
            sections = line.split('\t')
            if 'gene' in sections[2]:
                sections_2 = sections[8].split(';')
                for info in sections_2:
                    if 'CFD26' in sections_2[2]:
                        if sections_2[2][5:-1] in dct_num_key.values():
                            continue
                        else:
                            dct_num_key[c] = sections_2[1][5:-1]
                            dct_num_values[sections_2[1][5:-1]] = c
                            c +=1
                    else:
                        if 'CFD26' in sections_2[4]:
                            if sections_2[4][10:-1] in dct_num_key.values():
                                continue
                            else:
                                dct_num_key[c] = sections_2[4][10:-1]
                                dct_num_values[sections_2[4][10:-1]] = c
                                c +=1

In [ ]:
def get_percent_from_mafft(path):
    
    ids = []
    percent = []
    legnth = []
    
    os.chdir(path)
    
    for i in os.listdir():
        if '.txt' in i:
            with open(i, 'r') as f:
                #print(i)
                content = f.read()
                t = content.strip().split('>')
                
                af293 = t[1][13:]
                a1163 = t[2][12:]
                oerl = t[3][26:]
                fischeri = t[4][12:]
                lentulus = t[5][9:]
                affinis = t[6][6:]
                novo = t[7][17:]
                udagawae = t[8][9:]
                turcosus = t[9][13:]
                thermo = t[10][13:]

                count = 0
                for i in af293:
                    if i != '-':
                        count += 1
                for i in a1163:
                    if i != '-':
                        count += 1
                for i in oerl:
                    if i != '-':
                        count += 1
                for i in fischeri:
                    if i != '-':
                        count += 1
                for i in lentulus:
                    if i != '-':
                        count += 1
                for i in affinis:
                    if i != '-':
                        count += 1
                for i in novo:
                    if i != '-':
                        count += 1
                for i in udagawae:
                    if i != '-':
                        count += 1
                for i in turcosus:
                    if i != '-':
                        count += 1
                for i in thermo:
                    if i != '-':
                        count += 1
                        
                ids.append(t[1][:12])
                legnth.append(len(af293))
                percent.append(count/(len(af293)*10))
                
    names = 'gene legnth percent'.split()

    df = pd.DataFrame(columns=names)

    df['gene'] = ids
    df['legnth'] = legnth
    df['percent'] = percent

    return df

In [ ]:
#scripts to get column by column similarity 
def get_snps(path):
    
    gene_1_names = []
    gene_2_names = []
    snps_lst = []
    
    os.chdir(path)
    
    for file in os.listdir():
        if '.txt' in file:
            with open(file, 'r') as f:
                column_stats = []
                
                content = f.read()
                content_split = content.split('\n')

                gene_1_names.append(content_split[0][1:])
                gene_2_names.append(content_split[6][1:])
                
                seq_1 = content_split[1]
                seq_2 = content_split[7]
                
                seq_1_reverse = seq_1[::-1]
                seq_2_reverse = seq_2[::-1]
                
                if len(seq_1_reverse) <= len(seq_2_reverse):
                    len_total = len(seq_1_reverse)
                else:
                    len_total = len(seq_2_reverse)
                    
                for i in range(len_total):
                    if seq_1_reverse[i] == seq_2_reverse[i]:
                        column_stats.append(int(1))
                    if seq_1_reverse[i] != seq_2_reverse[i]:
                        column_stats.append(int(0))
                    else:
                        continue
                        
                snps_lst.append(column_stats)
            
    column_names = 'af293 fischeri snps'.split()
    df = pd.DataFrame(columns=column_names)
    
    df['af293'] = gene_1_names
    df['fischeri'] = gene_2_names
    df['snps'] = snps_lst
    
    return df